# TF-IDF Analysis

TF-IDF (term frequency-inverse document frequency) analysis is a statistical technique used in natural language processing and information retrieval to determine the importance of a word in a document or corpus. It is a way to measure how relevant a word is to a document in a collection of documents.

TF-IDF analysis assigns a weight to each word in a document based on how frequently it appears in the document (term frequency) and how rare it is in the entire corpus (inverse document frequency). The weight assigned to a word increases proportionally with its frequency in the document, but is offset by the rarity of the word in the corpus. This means that words that appear frequently in a document but also appear frequently in many other documents in the corpus are given a lower weight, while words that appear less frequently in the corpus but frequently in a particular document are given a higher weight.

The output of TF-IDF analysis is a numerical representation of each document that captures the importance of each word in that document. This can be used for various tasks such as text classification, clustering, and information retrieval.

## Table of Contents
* [Connect to Database ](#Connect-to-database)
* [Import Datasets](#Import-Dataset)
* [Remove Stopwords](#Remove-stopwords)
* [Lemmatization](#Lemmatization)

In [1]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
import spacy
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from numpy import argsort
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.evaluation import *
from Functions.NLP.alertp1_nlp import *
from Functions.NLP.data_with_nlp import *
from Functions.Pipelines import pipeline_NLP as NLP
#from Functions.Pipelines import *
from gensim.models import Word2Vec
import re
import nltk
import gensim

## Connect to Database

In [2]:
import mysql.connector
import pandas as pd

#creds = ["username","password","juliehaegh","ninG20&19rea","3306"] 
creds = ["juliehaegh","ninG20&19rea","172.20.20.4","hgo",3306]

In [3]:
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
275


## Import Datasets

In [4]:
# Import Alert P1 dataset
SClinic = pd.read_sql("""SELECT * FROM ConsultaUrgencia_doentespedidosconsultaNeurologia2012""",mydb)

# Import SClinic
AlertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)

# Replace all NaN with 0
AlertP1 = AlertP1.fillna(0)

# Add result column
AlertP1['result'] = ['Accepted' if x in [0,14,25,20,53,8,12,12] else 'Refused' for x in AlertP1['COD_MOTIVO_RECUSA']]

/var/folders/s4/b954cnz56f51x2w2tygb_fvc0000gn/T/ipykernel_35452/676277896.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SClinic = pd.read_sql("""SELECT * FROM ConsultaUrgencia_doentespedidosconsultaNeurologia2012""",mydb)
/var/folders/s4/b954cnz56f51x2w2tygb_fvc0000gn/T/ipykernel_35452/676277896.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AlertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)


In [5]:
data = NLP.pre_process(AlertP1)

/Users/juliehaegh/Documents/NovaSBE/PBL/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/juliehaegh/Documents/NovaSBE/PBL/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'
/Users/juliehaegh/Documents/NovaSBE/PBL/Functions/AlertP1/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [6]:
data = data[data['Texto']!='']

In [7]:
import math

# Split data into train and test
AlertP1_sorted = data[data['Texto']!=''].sort_values(by='DATA_RECEPCAO')

# calculate the index for the split
split_index = math.ceil(0.8 * len(AlertP1_sorted))

# split the data frame into test and train sets
train_set = AlertP1_sorted.iloc[:split_index]
test_set = AlertP1_sorted.iloc[split_index:]

In [8]:
# Import librariers 
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings(action="ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliehaegh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliehaegh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Remove Stopwords

In [9]:
# Get rid of special characters and transform Texto column to Latin words
train_set['Texto'] = train_set['Texto'].apply(lambda x: unidecode(x))

#The re.sub function is used to substitute all digits (\d) with an empty string
train_set['Texto'] = train_set['Texto'].apply(lambda x: re.sub(r'\d', '', x))

# Remove all names in Texto variable
# This function uses a regular expression to find all words in the text that start with a 
# capital letter (\b[A-Z][a-z]+\b), which are assumed to be names
text = train_set['Texto'] 

# remove all hyphens from the text
text = text.replace('-', '')

def remove_names(text):
    # Find all words that start with a capital letter
    names = re.findall(r'\b[A-Z][a-z]+\b', text)
    
    # Replace the names with an empty string
    for name in names:
        text = text.replace(name, '')
        
    return text

In [10]:
# Create an empty list to store the text
text_list = []

# Loop through the 'text' column
for text in text.str.lower(): # Transform every word to lower case
    text_list.append(text)

# Print the list of text
#print(text_list)

In [11]:
# Download the Portuguese stop words
nltk.download('stopwords')
nltk.download('punkt')

# Get the Portuguese stop words
stop_words = set(stopwords.words('portuguese'))

# Manually remove stopwords
stop_words.update(['-//','.', ',','(',')',':','-','?','+','/',';','2','1','drª','``','','3','desde','anos','doente','consulta','alterações','se',"''",'cerca','refere','hgo','utente','vossa','s','...','ainda','c','filha','costa','dr.','pereira','ja','--','p','dr','h','n','>','q','//','..','b','++','%','//','-','+++/','=','+++/'])

# Create a new list to store the filtered text
filtered_text = []

# Loop through the text_list and remove the stop words
for text in text_list:
    words = word_tokenize(text)
    words = [word for word in words if word.lower() not in stop_words]
    filtered_text.append(" ".join(words))

# Print the filtered text
#print(filtered_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliehaegh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliehaegh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# Save the filtered text as a new column to the dataframe
train_set['filtered_text'] = filtered_text

## Lemmatization

Lemmatization is a text normalization technique used in Natural Language Processing (NLP), that switches any kind of a word to its base root mode. Lemmatization is responsible for grouping different inflected forms of words into the root form, having the same meaning

In [13]:
# Define function for lemmatization
def spacy_lemmatizer(data):
    import spacy
    import pt_core_news_md
    nlp = pt_core_news_md.load()

    doclist = list(nlp.pipe(data))

    docs=[]
    for i, doc in enumerate(doclist):
        docs.append(' '.join([listitem.lemma_ for listitem in doc]))
        
    return docs

In [14]:
# create an empty list to store the words
word_list = []

# loop through each row of the "text_column" column
for index, row in train_set.iterrows():
    
    # split the text into individual words using whitespace as a delimiter
    words = row['filtered_text'].split()
    # add the words to the word list
    word_list.extend(words)

# print the word list
#print(word_list)

In [15]:
# create an empty list to store the words
word_list = []

# loop through each row of the "text_column" column
for index, row in train_set.iterrows():
    
    # split the text into individual words using whitespace as a delimiter
    words = row['filtered_text'].split()
    
    # remove hyphens from the words and add them to the word list
    word_list.extend([word.replace('-', '') for word in words])
    # remove slash from the words and ass them to the list
    word_list.extend([word.replace('/', '') for word in words])

In [16]:
Lemma = spacy_lemmatizer(word_list) # Call lemmatizer function

# print length of word_list and compare the count after doing lemmatization
from collections import Counter

items = Counter(Lemma).keys()
print('The number of words after lemmatization:',len(items))

items2 = Counter(word_list).keys()
print('The number of words before lemmatization:',len(items2))

The number of words after lemmatization: 8101
The number of words before lemmatization: 9885


In [17]:
# apply the spacy_lemmatizer function to each row in the 'text' column
train_set['text_lemmatized'] = spacy_lemmatizer(train_set['filtered_text'])

# drop rows with empty strings
train_set_filtered = train_set[['text_lemmatized','filtered_text','result']].replace('', pd.NA).dropna()
train_set_filtered = pd.DataFrame(train_set_filtered)
train_set_filtered

,text_lemmatized,filtered_text,result
630,idade tremor acentuar membro alto agravamento ...,idade tremor acentuado membros superiores agra...,1
1537,relatorio clinico,relatorio clinico,0
985,trazer inumero relatorio ida hgo-su vulvo-vagi...,traz inumeros relatorios idas hgo-su vulvo-vag...,1
1103,tremor fino ambos mao bocar ha mês deixar cair...,tremor fino ambas maos boca ha meses deixa cai...,1
752,pe pendente instalacao ha mês atrofiar muscula...,pe pendente instalacao ha meses atrofia muscul...,1
...,...,...,...
471,dte sii episodio recorrente lipotimer alteraco...,dte sii episodios recorrentes lipotimia altera...,0
1498,problema saude resolver demencia se enil / alz...,problema saude resolver demencia senil/alzheim...,1
244,antecedente epilepsia + ultimo ano crise frequ...,antecedentes epilepsia +ultimo ano crises freq...,0
1118,mulher idade diagnosticos dislipidemia hipertr...,mulher idade diagnosticos dislipidemia hipertr...,1


In [18]:
# Create a lambda function to apply to the DataFrame
train_set_filtered['accepted/rejected'] = train_set_filtered['result'].apply(lambda x: 1 if x == 'Accepted' else 0)

In [19]:
# Import Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets with 20% test size
X_train, X_test, y_train, y_test = train_test_split(train_set_filtered['filtered_text'], train_set_filtered['accepted/rejected'], test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer object with max_df = 0.8 and min_df = 5
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5)

# Fit and transform the training data into a sparse matrix of TF-IDF features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data into a sparse matrix of TF-IDF features
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Use the TF scores for future predictions
tf_vectorizer = TfidfVectorizer(use_idf=False, max_df=0.8, min_df=5)
X_tf = tf_vectorizer.fit_transform(train_set_filtered['filtered_text'])

# Use the TF-IDF scores for logistic regression with other variables
X_train_tfidf_lr = pd.concat([pd.DataFrame(X_train_tfidf.toarray()), train_set_filtered[train_set_filtered.index.isin(X_train.index)].drop(['filtered_text', 'accepted/rejected'], axis=1)], axis=1)
X_test_tfidf_lr = pd.concat([pd.DataFrame(X_test_tfidf.toarray()), train_set_filtered[train_set_filtered.index.isin(X_test.index)].drop(['filtered_text', 'accepted/rejected'], axis=1)], axis=1)

max_df is a float in the range [0.0, 1.0] or an integer, representing the maximum frequency of a term allowed in the corpus. Terms with a frequency higher than this threshold will be ignored as they are considered too common to be informative. In the code provided, we set max_df=0.8, which means that we are ignoring terms that appear in more than 80% of the documents in the corpus.

min_df is also a float in the range [0.0, 1.0] or an integer, representing the minimum frequency of a term allowed in the corpus. Terms with a frequency lower than this threshold will be ignored as they are considered too rare to be informative. In the code provided, we set min_df=5, which means that we are ignoring terms that appear in less than 5 documents in the corpus

In [20]:
feature_names = tfidf_vectorizer.get_feature_names_out()
len(feature_names)

1554


# Test

In [21]:
# Split the data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_set_filtered['filtered_text'], train_set_filtered['result'], test_size=0.2, random_state=42)

# Create the TF-IDF vectorizer object
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5)

# Fit and transform the vectorizer on the train set
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Extract the top 10 words with the highest scores for the train set
train_scores = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
train_features = tfidf_vectorizer.get_feature_names_out()
top_train_indices = np.argsort(train_scores)[::-1][:20]
top_train_words = [train_features[i] for i in top_train_indices]
top_train_scores = [train_scores[i] for i in top_train_indices]

# Transform the vectorizer on the test set
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Extract the top 10 words with the highest scores for the test set
test_scores = np.asarray(X_test_tfidf.mean(axis=0)).ravel().tolist()
test_features = tfidf_vectorizer.get_feature_names_out()
top_test_indices = np.argsort(test_scores)[::-1][:20]
top_test_words = [test_features[i] for i in top_test_indices]
top_test_scores = [test_scores[i] for i in top_test_indices]

# Print the results
print("Top 20 words with the highest scores for the train set:")
for word, score in zip(top_train_words, top_train_scores):
    print(f"{word}: {score:.4f}")

print("\nTop 20 words with the highest scores for the test set:")
for word, score in zip(top_test_words, top_test_scores):
    print(f"{word}: {score:.4f}")

Top 20 words with the highest scores for the train set:
relatorio: 0.0733
clinico: 0.0701
epilepsia: 0.0313
nao: 0.0304
alteracoes: 0.0274
ha: 0.0256
neurologia: 0.0219
demencia: 0.0207
mg: 0.0190
tac: 0.0181
se: 0.0180
tremor: 0.0175
cefaleias: 0.0169
quadro: 0.0168
avaliacao: 0.0167
ce: 0.0165
hta: 0.0162
memoria: 0.0155
agravamento: 0.0152
antecedentes: 0.0149

Top 20 words with the highest scores for the test set:
relatorio: 0.0893
clinico: 0.0866
nao: 0.0324
tremor: 0.0302
alteracoes: 0.0292
epilepsia: 0.0269
ha: 0.0269
se: 0.0258
memoria: 0.0240
demencia: 0.0217
tac: 0.0210
marcha: 0.0197
antecedentes: 0.0188
mg: 0.0184
ce: 0.0180
informacao: 0.0169
neurologia: 0.0168
hta: 0.0157
meses: 0.0157
dra: 0.0153


In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_top_words(train_set_filtered, num_words=20):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        train_set_filtered['filtered_text'],
        train_set_filtered['result'],
        test_size=0.2,
        random_state=42
    )

    # Create the TF-IDF vectorizer object
    tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5)

    # Fit and transform the vectorizer on the train set
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

    # Extract the top words with the highest scores for the train set
    train_scores = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
    train_features = tfidf_vectorizer.get_feature_names_out()
    top_train_indices = np.argsort(train_scores)[::-1][:num_words]
    top_train_words = [train_features[i] for i in top_train_indices]
    top_train_scores = [train_scores[i] for i in top_train_indices]

    # Transform the vectorizer on the test set
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # Extract the top words with the highest scores for the test set
    test_scores = np.asarray(X_test_tfidf.mean(axis=0)).ravel().tolist()
    test_features = tfidf_vectorizer.get_feature_names_out()
    top_test_indices = np.argsort(test_scores)[::-1][:num_words]
    top_test_words = [test_features[i] for i in top_test_indices]
    top_test_scores = [test_scores[i] for i in top_test_indices]

    # Return the top words and scores for both train and test sets
    return {
        'train': {
            'words': top_train_words,
            'scores': top_train_scores
        },
        'test': {
            'words': top_test_words,
            'scores': top_test_scores
        }
    }

In [23]:
# Assuming you have the `train_set_filtered` DataFrame
result = extract_top_words(train_set_filtered, num_words=20)

print("Top 20 words with the highest scores for the train set:")
for word, score in zip(result['train']['words'], result['train']['scores']):
    print(f"{word}: {score:.4f}")

print("\nTop 20 words with the highest scores for the test set:")
for word, score in zip(result['test']['words'], result['test']['scores']):
    print(f"{word}: {score:.4f}")

Top 20 words with the highest scores for the train set:
relatorio: 0.0733
clinico: 0.0701
epilepsia: 0.0313
nao: 0.0304
alteracoes: 0.0274
ha: 0.0256
neurologia: 0.0219
demencia: 0.0207
mg: 0.0190
tac: 0.0181
se: 0.0180
tremor: 0.0175
cefaleias: 0.0169
quadro: 0.0168
avaliacao: 0.0167
ce: 0.0165
hta: 0.0162
memoria: 0.0155
agravamento: 0.0152
antecedentes: 0.0149

Top 20 words with the highest scores for the test set:
relatorio: 0.0893
clinico: 0.0866
nao: 0.0324
tremor: 0.0302
alteracoes: 0.0292
epilepsia: 0.0269
ha: 0.0269
se: 0.0258
memoria: 0.0240
demencia: 0.0217
tac: 0.0210
marcha: 0.0197
antecedentes: 0.0188
mg: 0.0184
ce: 0.0180
informacao: 0.0169
neurologia: 0.0168
hta: 0.0157
meses: 0.0157
dra: 0.0153


In [24]:
def extract_top_words(train_set_filtered, num_words=20):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        train_set_filtered['filtered_text'],
        train_set_filtered['result'],
        test_size=0.2,
        random_state=42
    )

    # Create the TF-IDF vectorizer object
    tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=5)

    # Fit and transform the vectorizer on the train set
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

    # Extract the top words with the highest scores for the train set
    train_scores = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
    train_features = tfidf_vectorizer.get_feature_names_out()
    top_train_indices = np.argsort(train_scores)[::-1][:num_words]
    top_train_words = [train_features[i] for i in top_train_indices]
    top_train_scores = [train_scores[i] for i in top_train_indices]

    # Transform the vectorizer on the test set
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # Extract the top words with the highest scores for the test set
    test_scores = np.asarray(X_test_tfidf.mean(axis=0)).ravel().tolist()
    test_features = tfidf_vectorizer.get_feature_names_out()
    top_test_indices = np.argsort(test_scores)[::-1][:num_words]
    top_test_words = [test_features[i] for i in top_test_indices]
    top_test_scores = [test_scores[i] for i in top_test_indices]

    # Return the top words and scores for both train and test sets
    return {
        'train': {
            'words': top_train_words,
            'scores': top_train_scores
        },
        'test': {
            'words': top_test_words,
            'scores': top_test_scores
        }
    }

# Assuming you have the `train_set_filtered` DataFrame
num_words = 20
top_words = extract_top_words(train_set_filtered, num_words=num_words)

# Create a new DataFrame to store the extracted features
extracted_features = pd.DataFrame()

# Iterate over each row in the original dataset
for index, row in train_set_filtered.iterrows():
    word_scores = []
    # Compute the word scores for the top words
    for word in top_words['train']['words']:
        # Compute the word frequency or TF-IDF score for the current row
        score = row['filtered_text'].count(word)  # Example: Using word frequency
        word_scores.append(score)
    # Append the word scores to the new DataFrame
    extracted_features = extracted_features.append(pd.Series(word_scores), ignore_index=True)

In [25]:
extracted_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,1,2,0,0,1,0,0,0,3,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1281,0,0,0,0,0,1,3,4,1,1,4,0,0,1,2,2,0,3,3,0
1282,0,0,1,0,0,0,0,0,1,0,4,0,0,0,1,3,0,0,0,1
1283,0,0,0,3,3,3,0,0,0,0,9,0,0,1,0,6,1,1,0,0


The numbers in the example represent the word scores for the top words extracted from the text data. Each row corresponds to a document or instance in the dataset, and each column represents a specific word.

This row indicates the word scores for the top words in the document. Each number represents the score of the corresponding word at the same index. In this case, the top words are represented by their indices from 0 to 19.

For example, the score for word 0 is 0, the score for word 1 is 0, the score for word 2 is 0, and so on. The score for word 4 is 1, indicating that it appears once in the document.

Similarly, the following rows represent the word scores for other documents in the dataset. Each row represents a different document, and the numbers in the row indicate the scores for the corresponding words in that document.